# **DASHBOARD**

## **Introdução**

Este notebook é uma continuação da [análise da carteira](https://github.com/gabrielnrt/Finance/blob/main/AnaliseCarteira.ipynb) feita anteriormente. A ideia aqui é construir um painel interativo que mostre, através de gráficos de linhas, as variações percentual e total de todos os ativos da carteira, seguido de uma tabela contendo essas variações nos últimos dias.

Todos os dados estão contidos no arquivo `tabelafinal.csv`.

## **Dados Iniciais**

Vejamos primeiramente como estão dispostos os dados da tabela.

In [1]:
from pandas import read_csv, DataFrame

df = read_csv('tabelafinal.csv')

df.head()

,Date,Variação Total (R$),Variação Percentual,Nome
0,2021-06-25,-0.000002,-2.477076e-08,ITUB4
1,2021-06-28,-0.699998,-1.136361e-02,ITUB4
2,2021-06-29,-1.300001,-2.110391e-02,ITUB4
3,2021-06-30,-2.000002,-3.246756e-02,ITUB4
4,2021-07-01,-2.219999,-3.603894e-02,ITUB4


Tipos de dados:

In [4]:
df.dtypes

Date                    object
Variação Total (R$)    float64
Variação Percentual    float64
Nome                    object
dtype: object

Note que a data está como string. A priori, isso não será um problema para realizar o gráfico de uma série temporal pois ela está no formato aaaa-mm-dd, que é compatível com a biblioteca `plotly.express`.

Agora vejamos quais são os ativos que compõe essa carteira.

In [5]:
df['Nome'].unique()

array(['ITUB4', 'PETR4', 'VALE3', 'Carteira'], dtype=object)

Para separar o histórico de cada ativo, basta usar o método `loc` do pandas.

In [6]:
for ativo in df['Nome'].unique():

    print(df.loc[df['Nome']==ativo].head())
    print('\n')
    #break

         Date  Variação Total (R$)  Variação Percentual   Nome
0  2021-06-25            -0.000002        -2.477076e-08  ITUB4
1  2021-06-28            -0.699998        -1.136361e-02  ITUB4
2  2021-06-29            -1.300001        -2.110391e-02  ITUB4
3  2021-06-30            -2.000002        -3.246756e-02  ITUB4
4  2021-07-01            -2.219999        -3.603894e-02  ITUB4


           Date  Variação Total (R$)  Variação Percentual   Nome
310  2021-04-05             0.000007         3.808350e-08  PETR4
311  2021-04-06            -0.159996        -8.319277e-04  PETR4
312  2021-04-07            -0.320000        -1.663894e-03  PETR4
313  2021-04-08            -2.719994        -1.414306e-02  PETR4
314  2021-04-09            -3.120003        -1.622298e-02  PETR4


           Date  Variação Total (R$)  Variação Percentual   Nome
677  2021-05-13            -0.000006        -1.899040e-08  VALE3
678  2021-05-14            -5.790007        -1.715710e-02  VALE3
679  2021-05-17             2.909

## **Construção do painel**

Conforme dito na introdução, a estrutura do nosso painel será composta por um título, seguido de uma mensagem dizendo para selecionar um ativo da carteira. Depois dessa mensagem, há um Dropdown contendo as opções que o usuário pode selecionar, e com isso ele obterá seu respectivo gráfico da variação total, seguido da variação percentual e de uma tabela com o histórico desses valores nos últimos 10 dias.

In [4]:
from dash import Dash, html, dcc, Input, Output

lista = list(df['Nome'].unique())

app = Dash(__name__)

app.layout = html.Div(children = [
                html.H1('Análise Exploratória de uma carteira'),

                html.Div('Selecione um ativo'),

                html.Br(),

                dcc.Dropdown(id = 'botao',
                             options = lista,
                             value = 'Carteira'),

                dcc.Graph(id = 'graficoAbs',
                          figure = {}),

                html.Br(),

                dcc.Graph(id = 'graficoPct',
                          figure = {}),

                html.Br(),

                html.Table(id = 'tabela',
                           children = [])

])

## **Callbacks**

Com relação às callbacks, o parâmetro `value` do Dropdown vai ser o nosso input que mudará os parâmetros `figure` dos gráficos e o `children` da tabela, que foi baseada [neste tutorial](https://dash.plotly.com/layout#reusable-components) da própria documentação do plotly.

In [7]:
@app.callback(
    [Output('graficoAbs', 'figure'), Output('graficoPct', 'figure'), Output('tabela', 'children')],
    Input('botao', 'value')
)
def FuncaoAtualizadora(ativo):

    sub_df = df.loc[ df['Nome'] == ativo ]

    figuraAbs = line(data_frame = sub_df,
                     x = 'Date',
                     y = 'Variação Total (R$)')

    figuraPct = line(data_frame = sub_df,
                     x = 'Date',
                     y = 'Variação Percentual')

    sub_df2 = sub_df.tail(10)

    listaa = [
                html.Thead( html.Tr([html.Th(col) for col in sub_df2.columns])),
                html.Tbody([
                    html.Tr([
                        html.Td(sub_df2.iloc[i][col]) for col in sub_df2.columns
                    ]) for i in range(min(len(sub_df2), 10))
                ])
            ]

    return figuraAbs, figuraPct, listaa


Obs: tive que criar um segundo `sub_df` pois a tabela mostraria as 10 primeiras linhas caso contrário.

Para ver o painel, basta executar o código abaixo. Mas como dito em outros notebooks, é melhor fazer isso num script, que neste caso é o arquivo `painel.py`.

In [ ]:
if __name__ == '__main__':
    app.run_server(debug = True)